In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import StandardScaler
import ee
import geopandas as gpd
import json
import geemap
import numpy as np


def initialize_earth_engine():
    try:
        ee.Initialize()
    except Exception as e:
        ee.Authenticate()
        ee.Initialize()


def get_climate_data(start_date, end_date):

    watershed_shapefile = "Black_river.shp"  
    watershed_gdf = gpd.read_file(watershed_shapefile)


    watershed_geojson_str = watershed_gdf.to_json()
    watershed_geojson = json.loads(watershed_geojson_str)
    watershed = geemap.geojson_to_ee(watershed_geojson)


    datasets = {
        "PRISM_Precipitation": {
            "collection": "OREGONSTATE/PRISM/AN81d",
            "bands": ["ppt"]
        },
        "PRISM_Temperature_Max": {
            "collection": "OREGONSTATE/PRISM/AN81d",
            "bands": ["tmax"]
        },
        "PRISM_Temperature_Min": {
            "collection": "OREGONSTATE/PRISM/AN81d",
            "bands": ["tmin"]
        },
        "GRIDMET_Wind_Speed": {
            "collection": "IDAHO_EPSCOR/GRIDMET",
            "bands": ["vs"]
        },
        "GRIDMET_Relative_Humidity": {
            "collection": "IDAHO_EPSCOR/GRIDMET",
            "bands": ["rmin", "rmax"]
        },
        "GRIDMET_Solar_Radiation": {
            "collection": "IDAHO_EPSCOR/GRIDMET",
            "bands": ["srad"]
        }
    }


    def process_dataset(dataset_name, dataset_info):
        collection = ee.ImageCollection(dataset_info["collection"]) \
            .filterDate(start_date, end_date) \
            .select(dataset_info["bands"])

        collection_clipped = collection.map(lambda image: image.clip(watershed))

        def extract_daily_data(image):
            results = {}
            date = image.date().format("YYYY-MM-dd")
            results["date"] = date

            for band in dataset_info["bands"]:
                mean_value = image.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=watershed.geometry(),
                    scale=4000,
                    maxPixels=1e9
                ).get(band)
                results[band] = mean_value
            return ee.Feature(None, results)

        daily_fc = collection_clipped.map(extract_daily_data)
        daily_list = daily_fc.getInfo()["features"]

        daily_data = pd.DataFrame([
            {key: feature["properties"].get(key, None) for key in ["date"] + dataset_info["bands"]}
            for feature in daily_list
        ])

        daily_data["date"] = pd.to_datetime(daily_data["date"])
        daily_data.set_index("date", inplace=True)

        if dataset_name == "PRISM_Precipitation":
            monthly_data = daily_data.resample("ME").sum().reset_index()
        else:
            monthly_data = daily_data.resample("ME").mean().reset_index()

        if dataset_name == "GRIDMET_Relative_Humidity":
            monthly_data["mean_relative_humidity"] = (monthly_data["rmin"] + monthly_data["rmax"]) / 2 / 100
        elif dataset_name == "GRIDMET_Solar_Radiation":
            monthly_data["srad"] = monthly_data["srad"] * 0.0864

        return monthly_data

    merged_data = None
    for dataset_name, dataset_info in datasets.items():
        dataset_monthly = process_dataset(dataset_name, dataset_info)

        if merged_data is None:
            merged_data = dataset_monthly
        else:
            merged_data = pd.merge(merged_data, dataset_monthly, on="date", how="outer")

    column_order = [
        "date", "ppt", "tmax", "tmin", "srad", "vs", "mean_relative_humidity"
    ]
    merged_data = merged_data[column_order]
    return merged_data


class StreamflowApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Streamflow Simulation in the Black River Watershed")

        self.create_widgets()

    def create_widgets(self):
        frame = ttk.Frame(self.root, padding="10")
        frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

        ttk.Label(frame, text="Start Date (YYYY-MM-DD):").grid(row=0, column=0, sticky=tk.W)
        self.start_date_entry = ttk.Entry(frame, width=15)
        self.start_date_entry.grid(row=0, column=1)

        ttk.Label(frame, text="End Date (YYYY-MM-DD):").grid(row=1, column=0, sticky=tk.W)
        self.end_date_entry = ttk.Entry(frame, width=15)
        self.end_date_entry.grid(row=1, column=1)

        simulate_button = ttk.Button(frame, text="Simulate Streamflow", command=self.simulate_streamflow)
        simulate_button.grid(row=2, column=0, columnspan=2, pady=10)

        export_button = ttk.Button(frame, text="Export Data to Excel", command=self.export_data_to_excel)
        export_button.grid(row=3, column=0, columnspan=2, pady=10)

        ttk.Label(frame, text="Simulation Metrics:").grid(row=4, column=0, columnspan=2)

        self.metrics_text = tk.Text(frame, width=50, height=10, wrap=tk.WORD)
        self.metrics_text.grid(row=5, column=0, columnspan=2, pady=5)

        plot_button = ttk.Button(frame, text="Plot Results", command=self.plot_results)
        plot_button.grid(row=6, column=0, columnspan=2, pady=10)

    def simulate_streamflow(self):
        start_date = self.start_date_entry.get()
        end_date = self.end_date_entry.get()

        if not start_date or not end_date:
            messagebox.showerror("Input Error", "Please enter both start and end dates.")
            return

        try:
            climate_data = get_climate_data(start_date, end_date)

            model = joblib.load('RF_monthly.pkl')
            features = ["ppt", "tmax", "tmin", "srad", "vs", "mean_relative_humidity"]
            X = climate_data[features]

            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            y_simulated = model.predict(X_scaled)
            climate_data["Simulated_Streamflow"] = y_simulated

            max_flow = climate_data["Simulated_Streamflow"].max()
            min_flow = climate_data["Simulated_Streamflow"].min()
            mean_flow = climate_data["Simulated_Streamflow"].mean()

            self.metrics_text.delete(1.0, tk.END)
            self.metrics_text.insert(tk.END, f"Maximum Streamflow: {max_flow:.2f}\n")
            self.metrics_text.insert(tk.END, f"Minimum Streamflow: {min_flow:.2f}\n")
            self.metrics_text.insert(tk.END, f"Mean Streamflow: {mean_flow:.2f}\n")

            self.climate_data = climate_data

        except Exception as e:
            messagebox.showerror("Error", str(e))

    def export_data_to_excel(self):
        try:
            if hasattr(self, 'climate_data'):
                file_path = filedialog.asksaveasfilename(defaultextension=".xlsx",
                                                         filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])
                if file_path:
                    self.climate_data.to_excel(file_path, index=False)
                    messagebox.showinfo("Export Successful", "Data exported successfully to Excel.")
            else:
                messagebox.showerror("Error", "No data available to export. Please run the simulation first.")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def plot_results(self):
        try:
            if hasattr(self, 'climate_data'):
                plt.figure(figsize=(12, 6))
                plt.plot(self.climate_data["date"], self.climate_data["Simulated_Streamflow"], label="Simulated Streamflow", color="blue")
                plt.xlabel("Date")
                plt.ylabel("Streamflow")
                plt.title("Simulated Streamflow Over Time")
                plt.legend()
                plt.show(block=True)
            else:
                messagebox.showerror("Error", "No simulation data available. Please run the simulation first.")
        except Exception as e:
            messagebox.showerror("Error", str(e))

if __name__ == "__main__":
    initialize_earth_engine()
    root = tk.Tk()
    app = StreamflowApp(root)
    root.mainloop()